In [14]:
import pandas as pd
import numpy as np 
from tqdm import tqdm

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [15]:
num_games = 2
season_1819 = pd.read_csv(f'NewData/AVG(18-19)[{num_games}].csv').dropna().drop('Unnamed: 0', axis = 1).reset_index(drop= True)
season_1920 = pd.read_csv(f'NewData/AVG(19-20)[{num_games}].csv').dropna().drop('Unnamed: 0', axis = 1).reset_index(drop = True)




In [ ]:
def time_series_single(df, season):
    unique_players = df.Name.value_counts().index
    col =  ['MP', 'FG', 'FGA', '3P', '3PA',
            'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK',
            'PTS', 'FDP']
    t_col = ['FDP'] 
    #updater_column = ['Name', 'Date', 'FDP'] + avg_col
    timesteps = 10
    final_df = pd.DataFrame(columns = ['Name', 'Date', 'FDP'] + [str(i) for i in list(range(1,timesteps+1))[::-1]])
    pbar = tqdm(unique_players, desc = f'Single Feature: Converting DF to timeseries ({season})')
    for player in pbar: 
        new_df = df[df.Name == player].sort_values(by = ['Date']).reset_index(drop = True)
        for idx in range(len(new_df)):

            row = new_df.iloc[idx]

            if idx <= timesteps-1: 
                continue 
            last_games = new_df[idx-timesteps: idx]
            l_games = [] 
            for i in range(len(last_games)): 
                l_row = last_games.iloc[i]
                fdp = l_row[t_col].values[0]
                
                l_games.append(fdp)
            
            l_games = l_games[::-1]
            row_dict = {'Name': player, 'Date': row['Date'], 'FDP': row['FDP']}
            for i in range(timesteps): 
                t_dict = {str(i+1): l_games[i]}
                row_dict.update(t_dict)
            final_df = final_df.append(row_dict, ignore_index = True)
            
        
        
    final_df.to_csv(f'NewData/TimeSeriesSingleFeatures({season})[{num_games}].csv', index = False)
    return final_df
             
        
time_series_single_1819 = time_series_single(season_1819, '18-19')   
time_series_single_1920 = time_series_single(season_1920, '19-20')    


time_series_single_1920.head()

In [16]:
from sklearn.preprocessing import MinMaxScaler
def time_series_multiple(df, season):
    min_max = MinMaxScaler()
    
    unique_players = df.Name.value_counts().index
    col =  ['MP', 'FG', 'FGA', '3P', '3PA',
            'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK',
            'PTS', 'FDP']
    avg_col = ['FDP'] + [f'{num_games}_{i}' for i in col]
    updater_column = ['Name', 'Date', 'FDP'] + avg_col
    timesteps = 10
    final_df = pd.DataFrame(columns = ['Name', 'Date', 'FDP'] + [str(i) for i in list(range(1,timesteps+1))[::-1]])
    df['New_FDP'] = df.FDP.values
    df[avg_col] = min_max.fit_transform(df[avg_col].values)
    pbar = tqdm(unique_players, desc = f'Multiple: Converting DF to timeseries ({season})')
    for player in pbar:
        new_df = df[df.Name == player].sort_values(by = ['Date']).reset_index(drop = True)
        #print(new_df)
        for idx in range(len(new_df)):

            row = new_df.iloc[idx]

            if idx <= timesteps-1: 
                continue 
            last_games = new_df[idx-timesteps: idx]
            l_games = [] 
            for i in range(len(last_games)): 
                l_row = last_games.iloc[i]
                avg_array = np.array(l_row[avg_col])
                
                l_games.append(avg_array)
            
            l_games = l_games[::-1]
            row_dict = {'Name': player, 'Date': row['Date'], 'FDP': row['New_FDP']}#.update({'hello': 0})
            for i in range(timesteps): 
                t_dict = {str(i+1): l_games[i]}
                row_dict.update(t_dict)
            final_df = final_df.append(row_dict, ignore_index = True)
            
        
            
        
            
        
    final_df.to_csv(f'NewData/TimeSeriesMultFeaturesScaled({season})[{num_games}].csv', index = False)
    return final_df
             
        
time_series_multiple_1819 = time_series_multiple(season_1819, '18-19')   
time_series_multiple_1920 = time_series_multiple(season_1920, '19-20')    

time_series_multiple_1819.head()

Multiple: Converting DF to timeseries (19-20): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 454/454 [04:03<00:00,  1.86it/s]


,Name,Date,FDP,10,9,8,7,6,5,4,3,2,1
0,Pascal Siakam,20181110.0,27.6,"[0.32653061224489793, 0.12482422931314223, 0.0...","[0.14285714285714285, 0.3619253650621958, 0.15...","[0.30918367346938774, 0.3598702001081666, 0.15...","[0.26428571428571423, 0.16700919415900486, 0.1...","[0.5163265306122449, 0.19459167117360743, 0.15...","[0.4591836734693877, 0.4624121146565711, 0.375...","[0.23979591836734693, 0.459599783666847, 0.343...","[0.4091836734693877, 0.3857220118983234, 0.156...","[0.3204081632653061, 0.6570037858301785, 0.375...","[0.34795918367346934, 0.7017847485127096, 0.46..."
1,Pascal Siakam,20181112.0,39.1,"[0.14285714285714285, 0.3619253650621958, 0.15...","[0.30918367346938774, 0.3598702001081666, 0.15...","[0.26428571428571423, 0.16700919415900486, 0.1...","[0.5163265306122449, 0.19459167117360743, 0.15...","[0.4591836734693877, 0.4624121146565711, 0.375...","[0.23979591836734693, 0.459599783666847, 0.343...","[0.4091836734693877, 0.3857220118983234, 0.156...","[0.3204081632653061, 0.6570037858301785, 0.375...","[0.34795918367346934, 0.7017847485127096, 0.46...","[0.31224489795918364, 0.7039480800432666, 0.46..."
2,Pascal Siakam,20181114.0,33.4,"[0.30918367346938774, 0.3598702001081666, 0.15...","[0.26428571428571423, 0.16700919415900486, 0.1...","[0.5163265306122449, 0.19459167117360743, 0.15...","[0.4591836734693877, 0.4624121146565711, 0.375...","[0.23979591836734693, 0.459599783666847, 0.343...","[0.4091836734693877, 0.3857220118983234, 0.156...","[0.3204081632653061, 0.6570037858301785, 0.375...","[0.34795918367346934, 0.7017847485127096, 0.46...","[0.31224489795918364, 0.7039480800432666, 0.46...","[0.42959183673469387, 0.4842617631151975, 0.28..."
3,Pascal Siakam,20181116.0,36.8,"[0.26428571428571423, 0.16700919415900486, 0.1...","[0.5163265306122449, 0.19459167117360743, 0.15...","[0.4591836734693877, 0.4624121146565711, 0.375...","[0.23979591836734693, 0.459599783666847, 0.343...","[0.4091836734693877, 0.3857220118983234, 0.156...","[0.3204081632653061, 0.6570037858301785, 0.375...","[0.34795918367346934, 0.7017847485127096, 0.46...","[0.31224489795918364, 0.7039480800432666, 0.46...","[0.42959183673469387, 0.4842617631151975, 0.28...","[0.3714285714285714, 0.21060032449972957, 0.12..."
4,Pascal Siakam,20181117.0,34.1,"[0.5163265306122449, 0.19459167117360743, 0.15...","[0.4591836734693877, 0.4624121146565711, 0.375...","[0.23979591836734693, 0.459599783666847, 0.343...","[0.4091836734693877, 0.3857220118983234, 0.156...","[0.3204081632653061, 0.6570037858301785, 0.375...","[0.34795918367346934, 0.7017847485127096, 0.46...","[0.31224489795918364, 0.7039480800432666, 0.46...","[0.42959183673469387, 0.4842617631151975, 0.28...","[0.3714285714285714, 0.21060032449972957, 0.12...","[0.40612244897959177, 0.21124932395889673, 0.1..."
